Make sure the docker is properly launched on ip 127.0.0.1

list of available models (all same version for convenience):

6-lgbm
5-2-grid-search-model
6-2-nn-tuned-model
6-4-stacked-trained-tuned-model
3-3-angle-dist-logreg-model


In [1]:
import os


import ipywidgets as widgets
from ipywidgets import interact, interact_manual, widget, Layout, Button, HBox, FloatText, Textarea, Dropdown, Label, VBox
from IPython.display import display
from IPython.html import widgets
from prettytable import PrettyTable
from ift6758.ift6758.client.serving_client import ServingClient
from ift6758.ift6758.client.game_client import Game_Client
import pandas as pd
import numpy as np


form_item_layout = Layout(
    display='inline-flex',
    flex_flow='row',
    flex_wrap= 'wrap',
    justify_content='space-between',
    max_width='0.8'
)

#Creating required variable
prevgameid = None
gameid = widgets.Text()
GAME_ID_FLAG = 'GAME_ID_FLAG'
update_data = Button(description='Update Data', button_style='danger')
model = widgets.Text(value='6-lgbm')
version = widgets.Text(value = '1.0.1')
workspace= widgets.Text(value='axelbogos')
update_model = Button(description='Update Model', button_style='danger')
out = widgets.Output()
period = 1
time_left = "20:00"

game_data = pd.DataFrame()

serving_client = ServingClient(ip="127.0.0.1")
game_client = None

def update_game_ID(gameid=gameid, prevgameid=prevgameid, game_data=game_data):
    if gameid != prevgameid:
        game_client = Game_Client(game_id = int(float(gameid.value)))
        game_data, period, time_left = game_client.ping_api()
        if not game_data.empty:
            if game_data.shape[0] > 0:
                predictions = serving_client.predict(game_data)
                game_data['predict'] = predictions['0'].tolist()
                prevgameid = gameid
    else:
        game_data_temp, period, time_left = game_client.ping_api()
        if not game_data_temp.empty:
            if game_data_temp.shape[0] > 0:
                predictions = serving_client.predict(game_data_temp)
                game_data_temp['predict'] = predictions['0'].tolist()
                game_data.append(game_data_temp)
    if len(time_left.split(':')[1]) == 1:
        time_left = f"{time_left.split(':')[0]}:00"
    if period == 4:
        time_left = f"{int(time_left.split(':')[0])-15}:{time_left.split(':')[1]}"
    if period == 5:
        time_left = "0:00"
    return game_data, prevgameid, period, time_left

def create_dashboard(game_data=game_data):
    home_team = game_data.iloc[0]["home_team"]
    away_team = game_data.iloc[0]["away_team"]
    
    x = PrettyTable(["Home Team", "Away Team"])
    x.add_row([home_team, away_team])
    x.add_row([np.round(game_data.predict[game_data["team"]== game_data["home_team"]].sum(),3),
                np.round(game_data.predict[game_data["team"]==game_data["away_team"]].sum(),3)])
    return x        

def on_game_button_click(b):
    game_data, prevgameid, period, time_left = update_game_ID()
    if not game_data.empty:
        if game_data.shape[0] > 0:
            x=create_dashboard(game_data)
            
            with out:
                    out.clear_output()
                    
                    display(print(x.get_string(title=f"Period {period} - {time_left}")))
                    display(game_data)
    else:
        with out:
            out.clear_output()
            display('No data yet for this game')



def on_model_button_click(b):
    serving_client.download_registry_model(workspace.value,model.value,version.value)
    if not game_data.empty:
        if game_data.shape[0] > 0:
            predictions = serving_client.predict(game_data)
            game_data['predict'] = predictions['0'].tolist()
            x=create_dashboard(game_data)
    with out:
            out.clear_output()
            display('model updated')
            if not game_data.empty:
                if game_data.shape[0] > 0:
                    display(print(x.get_string(title=f"Period {period} - {str(time_left)}")))
                    display(game_data)
            else:
                display('No data yet for this game')

        

update_data.on_click(on_game_button_click)
update_model.on_click(on_model_button_click)

box1 = HBox([Label(value='Workspace',layout=Layout(display="flex")),
      workspace, Label(value='Model:'), model], layout=form_item_layout)
box2 =HBox([Label(value='version'), version, update_model], layout=form_item_layout)
box3 = HBox([Label(value='GameID'),
        gameid,update_data], layout=form_item_layout)
box4 = HBox([out],layout=form_item_layout)

VBox([box1,box2,box3,box4])

C:\Anaconda31\envs\6758\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
